# IMT.fi Funnel Analysis 2019-2024

Olli Salli, 2025

Note! A typical notebook would have commentary and more informative headings; these are OMITTED HERE to avoid leaking business details to those without access to the source data. 

In [ ]:
start_date = '2019-01-01'
end_date = '2024-12-31'

In [ ]:
from db_connection import configure_pandas_display, get_redshift_engine

configure_pandas_display()

engine = get_redshift_engine('imt_aws_redshift', 'dev')

%load_ext sql
%sql engine
%config SqlMagic.autopandas = True
%config SqlMagic.named_parameters = 'enabled'

In [ ]:
# Import plotting helpers
from funnel_plots import (
    plot_product_step_comparison,
    plot_completion_comparison,
    plot_abandonment_stages,
)

# Auto-reload modules when they change
%load_ext autoreload
%autoreload 2

## Overall reservation completion comparison

In [ ]:
%%sql --save res_by_year
SELECT
        departureyear,
        COUNT(*) AS total_reservations,
        SUM(CASE WHEN finalstage = 'Confirmed' THEN 1 ELSE 0 END) AS confirmed
    FROM dbt_osalli.fct_reservation_funnel
    WHERE departuredate BETWEEN :start_date AND :end_date
    GROUP BY departureyear
    ORDER BY departureyear;

In [ ]:
res_by_year_df = %sql SELECT * from res_by_year

plot_completion_comparison(res_by_year_df)

## Abandonment stages 2019 vs 2024
### Whole pipeline

In [ ]:
%%sql --save abandonment_stages
SELECT
    departureyear,
    finalstage,
    COUNT(*) AS count
FROM dbt_osalli.fct_reservation_funnel
WHERE departuredate BETWEEN :start_date AND :end_date
    AND finalstage != 'Confirmed'
    AND departureyear IN (2019, 2024)
GROUP BY departureyear, finalstage
ORDER BY departureyear, count DESC;

In [ ]:
abandonment_df = %sql SELECT * from abandonment_stages

plot_abandonment_stages(abandonment_df)

### Within ProductSelection

In [ ]:
%%sql --save product_selection_abandonment
SELECT
    departureyear,
    abandonproductsteptype,
    COUNT(*) AS count
FROM dbt_osalli.fct_reservation_funnel
WHERE departuredate BETWEEN :start_date AND :end_date
    AND departureyear IN (2019, 2024)
GROUP BY departureyear, abandonproductsteptype
ORDER BY departureyear, count DESC;

In [ ]:
prod_select_df = %sql SELECT * from product_selection_abandonment

plot_product_step_comparison(prod_select_df, 'ProductSelection Abandonment by Step: 2019 vs 2024')

#### By specific trip types

The above difference in the relative importance of ship, flight and hotel-related steps is mostly explained by changes in trip distribution. Let's make the comparison more apples-to-apples by comparing trips with similar structures next.

In [ ]:
%%sql --save res_by_abd_prod_step
SELECT
    departureyear,
    abandonproductsteptype,
    tripnumhotelphases,
    tripnumshipphases,
    tripnumflightphases,
    tripnumphases,
    COUNT(*) AS count
FROM dbt_osalli.fct_reservation_funnel
JOIN dbt_osalli.dim_trip USING (tripid)
WHERE departuredate BETWEEN :start_date AND :end_date
    AND departureyear IN (2019, 2024)
GROUP BY ALL;

There's surprisingly little change in the product selection stages where people abandon their Ship-Hotel-Ship reservations (e.g. Hotel trip to Tallinn via ferry):

In [ ]:
allpsabd_df = %sql SELECT * from res_by_abd_prod_step

# Filter for trips with 1 hotel phase and 2 ship phases
filtered_df = allpsabd_df[
    (allpsabd_df['tripnumhotelphases'] == 1) &
    (allpsabd_df['tripnumshipphases'] == 2)
]

# Sum pre-aggregated counts by year and step type for plotting
plot_data = filtered_df.groupby(['departureyear', 'abandonproductsteptype'], dropna=False)['count'].sum().reset_index()

plot_product_step_comparison(plot_data, 'ProductSelection Abandonment (Ship-Hotel-Ship): 2019 vs 2024')

For cruises (only Ship steps) there is even less change:

In [ ]:
allpsabd_df = %sql SELECT * from res_by_abd_prod_step

# Filter for trips where all phases are ships
filtered_df = allpsabd_df[
    allpsabd_df['tripnumshipphases'] == allpsabd_df['tripnumphases']
]

# Sum pre-aggregated counts by year and step type for plotting
plot_data = filtered_df.groupby(['departureyear', 'abandonproductsteptype'], dropna=False)['count'].sum().reset_index()

plot_product_step_comparison(plot_data, 'ProductSelection Abandonment (Ship only): 2019 vs 2024')

Trips with flights are nowadays more popular, and their abandons have shifted farther in the pipeline:

In [ ]:
allpsabd_df = %sql SELECT * from res_by_abd_prod_step

# Filter for trips with flight phases
filtered_df = allpsabd_df[
    allpsabd_df['tripnumflightphases'] > 0
]

# Sum pre-aggregated counts by year and step type for plotting
plot_data = filtered_df.groupby(['departureyear', 'abandonproductsteptype'], dropna=False)['count'].sum().reset_index()

plot_product_step_comparison(plot_data, 'ProductSelection Abandonment (Trips with flights): 2019 vs 2024')